<a href="https://colab.research.google.com/github/Carlosvergara1995/Problem_Set_4_Predicting_Tweets/blob/main/Analizador_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##TProblem_Set_4_Predicting_Tweets


##CARGA DE DATOS

In [ ]:
#IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
!pip install nltk
!python -m nltk.downloader stopwords
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.1 MB/s eta 0:00:00


# Carga de datos

In [ ]:
###Cargamos datos
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

df_test.shape


FileNotFoundError: ignored

In [ ]:
print(df_train.shape)
#Calculamos na
print("Los NA son:")
print(df_train.isna().sum())

No hay NA, por lo tanto no se elimina ningun valor, ni se realiza imputacion

In [ ]:
#Contamos número de registros para cada una de las personas
print("Datos de entrenamiento")
df_train.groupby('name')['text'].count()

# Preprocesamiento de los datos

Se encuentra que es una muestra relativamente equilibra, por lo tanto no se realizara ningun tipo de trabajo adicional respecto al balanceo.

In [ ]:
#Creamos una columna para trabajar el procesamiento
df_train = df_train.assign(texto_procesado=df_train.text)
df_test = df_test.assign(texto_procesado=df_test.text)

## Se transforman todas las palabras a minusculas
df_train.texto_procesado = df_train.texto_procesado.str.lower()
df_test.texto_procesado = df_test.texto_procesado.str.lower()

# Quitamos tildes 
from unidecode import unidecode
df_train.texto_procesado = df_train.texto_procesado.apply(lambda x: unidecode(x))
df_test.texto_procesado = df_test.texto_procesado.apply(lambda x: unidecode(x))

#  quitamos signos, aunque TfidfVectorizer los quita por defecto
df_train.texto_procesado = df_train.texto_procesado.str.replace('-', ' ')
df_test.texto_procesado = df_test.texto_procesado.str.replace('-', ' ')
df_train.texto_procesado = df_train.texto_procesado.str.replace(r'[^\w\s]', '')
df_test.texto_procesado = df_test.texto_procesado.str.replace(r'[^\w\s]', '')

#  quitamos los saltos que estan en forma de \n
df_train.texto_procesado = df_train.texto_procesado.str.replace('\n', '')
df_test.texto_procesado = df_test.texto_procesado.str.replace('\n', '')

In [ ]:
print("Texto sin procesar")
print(df_train.text[1])
print("Texto procesado")
print(df_train.texto_procesado[1])

Texto sin procesar
¡Si Bogotá y Cundinamarca salen adelante Colombia sale adelante! Trabajando juntos y reconociéndonos como iguales cundinamarqueses y bogotanos vamos lograr que nuestros campesinos tengan una vida digna. De eso se trata construir paz y así lo haremos en Bogotá-Región. 
Texto procesado
si bogota y cundinamarca salen adelante colombia sale adelante trabajando juntos y reconociendonos como iguales cundinamarqueses y bogotanos vamos lograr que nuestros campesinos tengan una vida digna de eso se trata construir paz y asi lo haremos en bogota region 


In [ ]:
#eliminamos las stopword del los textos

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#Palabras en espanol
stop_words = set(stopwords.words('spanish'))
stop_words.update(["comida", "restaurante", "lugar", "día","usted","ustedes","usd"])

#Creamos una funcion que nos permite eliminar las stopword de cada columna
def eliminar_stopwords(texto):
    palabras = word_tokenize(texto)
    palabras_sin_stopwords = [palabra for palabra in palabras if palabra not in stop_words]
    texto_sin_stopwords = ' '.join(palabras_sin_stopwords)
    return texto_sin_stopwords

#Aplicamos la funcion a cada fila
df_train.texto_procesado_sin = df_train.texto_procesado.apply(lambda x: eliminar_stopwords(x))
df_test.texto_procesado_sin = df_test.texto_procesado.apply(lambda x: eliminar_stopwords(x))


#Revisamos los resultados
print("Texto sin procesar")
print(df_train.text[1])
print("Texto procesado")
print(df_train.texto_procesado_sin[1])